In [52]:
import pandas as pd
import plotly.express as px


In [ ]:
OO = pd.read_parquet('data/REP_ORD_ORDER.parquet',engine = 'pyarrow',columns = ['ORDER_ID','ORDER_NUM','JOB_CODE','ELIGIBLE'])
JC = pd.read_parquet('data/REP_ORD_JOB_CODE.parquet',engine = 'pyarrow',columns = ['JOB_CODE_ID','NAME','CORE_DESCRIPTION'])
OOS = pd.read_parquet('data/REP_ORD_ORDER_STATE.parquet',engine = 'pyarrow',columns=['ORDER_STATE_ID','FOR_ORDER','ORDER_NUM','LATEST_ASSIGNMENT','TOTAL_TIME_EN_ROUTE','TOTAL_TIME_ON_SITE','ONSITE_AT', 'COMPLETED', 'ENROUTE_AT'])
AA = pd.read_parquet('data/REP_ASN_ASSIGNMENT.parquet',engine = 'pyarrow',columns = ['ASSIGNMENT_ID','FOR_RESOURCE'])
LR = pd.read_parquet('data/REP_LAB_RESOURCE.parquet',engine = 'pyarrow',columns = ['RESOURCE_ID','FOR_USER'])
LU = pd.read_parquet('data/REP_LAB_USER.parquet',engine = 'pyarrow',columns = ['USER_ID','LOGON_ID'])

,0,1,2
ORDER_ID,111000002906541,111000002928630,111000002928254
ORDER_NUM_x,SAP000010660531,SAP000010678862,SAP000010677526
JOB_CODE,211000000001080,211000000000452,211000000001080
ELIGIBLE,2022-06-24 15:00:00,2022-06-24 15:00:00,2022-06-24 15:00:00
JOB_CODE_ID,211000000001080,211000000000452,211000000001080
NAME,G37.02,E13.3.3,G37.02
CORE_DESCRIPTION,Fumigation - Turn Off Gas,E Mtr Chg 1 Ph Res 200,Fumigation - Turn Off Gas
ORDER_STATE_ID,369000002906541,369000002928630,369000002928254
FOR_ORDER,111000002906541,111000002928630,111000002928254
ORDER_NUM_y,SAP000010660531,SAP000010678862,SAP000010677526


In [ ]:
DF = OO.merge(JC,left_on = 'JOB_CODE',right_on='JOB_CODE_ID')
DF = DF.merge(OOS, left_on = 'ORDER_ID', right_on = 'FOR_ORDER')
DF = DF.merge(AA,left_on = 'LATEST_ASSIGNMENT',right_on = 'ASSIGNMENT_ID')
DF = DF.merge(LR,left_on = 'FOR_RESOURCE',right_on = 'RESOURCE_ID')
DF = DF.merge(LU,left_on = 'FOR_USER',right_on = 'USER_ID')

DF[:3].T

In [54]:
timestamp_cols = ['ELIGIBLE','ONSITE_AT','COMPLETED','ENROUTE_AT',]

DF[timestamp_cols] = DF[timestamp_cols].apply(pd.to_datetime)

In [55]:
DF['WORK_START'] = DF['ENROUTE_AT']  
DF['WORK_END'] = DF['COMPLETED']    
DF['ONSITE_START'] = DF['ONSITE_AT'] 

DF['ACTUAL_WORK_DURATION'] = (DF['WORK_END'] - DF['WORK_START']).dt.total_seconds() / 60

In [82]:
def get_work_dates(start_time, end_time):
    if pd.isna(start_time) or pd.isna(end_time):
        return []
    start_date = start_time.date()
    end_date = end_time.date()
    dates = []
    current_date = start_date
    while current_date <= end_date:
        dates.append(current_date)
        current_date += pd.Timedelta(days=1)
    return dates


expanded_records = []

for idx, row in DF.iterrows():
    if pd.notna(row['WORK_START']) and pd.notna(row['WORK_END']):
        work_dates = get_work_dates(row['WORK_START'], row['WORK_END'])
        if len(work_dates) == 1:
            expanded_records.append({
                'LOGON_ID': row['LOGON_ID'],
                'CORE_DESCRIPTION': row['CORE_DESCRIPTION'],
                'DATE': work_dates[0],
                'DAILY_WORK_TIME': row['ACTUAL_WORK_DURATION'],
                'ORDER_ID': row['ORDER_ID'],
                'JOB_CODE': row['NAME'],
                'WORK_START': row['WORK_START'],
                'WORK_END': row['WORK_END']
            })
        else:
            total_duration = row['ACTUAL_WORK_DURATION']
            daily_duration = total_duration / len(work_dates)
            
            for work_date in work_dates:
                expanded_records.append({
                    'LOGON_ID': row['LOGON_ID'],
                    'CORE_DESCRIPTION': row['CORE_DESCRIPTION'],
                    'DATE': work_date,
                    'DAILY_WORK_TIME': daily_duration,
                    'ORDER_ID': row['ORDER_ID'],
                    'JOB_CODE': row['NAME'],
                    'WORK_START': row['WORK_START'],
                    'WORK_END': row['WORK_END']
                })

expanded_df = pd.DataFrame(expanded_records)

In [83]:
expanded_df[:3].T

,0,1,2
LOGON_ID,04904,09699,04904
CORE_DESCRIPTION,Fumigation - Turn Off Gas,E Mtr Chg 1 Ph Res 200,Fumigation - Turn Off Gas
DATE,2022-06-24,2022-06-25,2022-06-24
DAILY_WORK_TIME,48.65,29.35,27.5
ORDER_ID,111000002906541,111000002928630,111000002928254
JOB_CODE,G37.02,E13.3.3,G37.02
WORK_START,2022-06-24 15:15:42,2022-06-25 00:36:46,2022-06-24 16:05:00
WORK_END,2022-06-24 16:04:21,2022-06-25 01:06:07,2022-06-24 16:32:30


In [ ]:
daily_utilization = expanded_df.groupby(['LOGON_ID', 'DATE']).agg({
    'DAILY_WORK_TIME': 'sum',
    'ORDER_ID': 'count'
}).reset_index()

daily_utilization.rename(columns={
    'DAILY_WORK_TIME': 'TOTAL_MINUTES_WORKED',
    'ORDER_ID': 'JOBS_COUNT'
}, inplace=True)

DAILY_WORK_MIN = 7 * 60  
daily_utilization['UTILIZATION_RATE'] = (
    daily_utilization['TOTAL_MINUTES_WORKED'] / DAILY_WORK_MIN
).round(4)

In [59]:
daily_utilization['UTILIZATION_RATE_%'] = daily_utilization['UTILIZATION_RATE'] *100
daily_utilization['DATE'] = daily_utilization['DATE'].apply(pd.to_datetime)
daily_utilization[:5].T

,0,1,2,3,4
LOGON_ID,00018,00018,00018,00018,00018
DATE,2019-06-11 00:00:00,2019-06-12 00:00:00,2019-07-30 00:00:00,2019-07-31 00:00:00,2019-10-21 00:00:00
TOTAL_MINUTES_WORKED,205.633333,25.383333,305.375,24.108333,85.15
JOBS_COUNT,16,1,6,2,2
UTILIZATION_RATE,0.4896,0.0604,0.7271,0.0574,0.2027
UTILIZATION_RATE_%,48.96,6.04,72.71,5.74,20.27


In [60]:
util_job_counts = daily_utilization.copy()

In [86]:
bins = [0, 3, 6, 9, 12, 15, 18, float('inf')]
labels = ['1–3', '4–6', '7–9', '10–12', '13–15', '16–18', '19+']
util_job_counts['JOB_COUNT_RANGE'] = pd.cut(util_job_counts['JOBS_COUNT'], bins=bins, labels=labels, right=True)

job_range_avg = (
    util_job_counts
    .groupby('JOB_COUNT_RANGE', observed=False)['UTILIZATION_RATE_%']
    .mean()
    .reset_index()
)

fig1 = px.bar(
    job_range_avg,
    x='JOB_COUNT_RANGE',
    y='UTILIZATION_RATE_%',
    text=job_range_avg['UTILIZATION_RATE_%'].round(1).astype(str) + '%',
    title='Average Utilization Rate by Job Counts',
    labels={'UTILIZATION_RATE_%': 'Average Utilization Rate (%)', 'JOB_COUNT_RANGE': 'Job Count Range'}
)

fig1.update_layout(template='simple_white')
fig1.show()

In [87]:
util_day_ofweek = daily_utilization.copy()

In [92]:
util_day_ofweek['DAY_OF_WEEK'] = util_day_ofweek['DATE'].dt.day_of_week

map_day = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}

util_day_ofweek['DAY_OF_WEEK'] = util_day_ofweek['DAY_OF_WEEK'].map(map_day)

In [ ]:
util_day_ofweek

,LOGON_ID,DATE,TOTAL_MINUTES_WORKED,JOBS_COUNT,UTILIZATION_RATE,UTILIZATION_RATE_%,DAY_OF_WEEK
0,00018,2019-06-11,205.633333,16,0.4896,48.96,Tuesday
1,00018,2019-06-12,25.383333,1,0.0604,6.04,Wednesday
2,00018,2019-07-30,305.375000,6,0.7271,72.71,Tuesday
3,00018,2019-07-31,24.108333,2,0.0574,5.74,Wednesday
4,00018,2019-10-21,85.150000,2,0.2027,20.27,Monday
...,...,...,...,...,...,...,...
188628,TK200,2019-09-26,3.500000,1,0.0083,0.83,Thursday
188629,TK200,2020-08-07,1.733333,1,0.0041,0.41,Friday
188630,TK223,2023-02-24,2.700000,1,0.0064,0.64,Friday
188631,TK513,2022-06-28,8.983333,1,0.0214,2.14,Tuesday


In [95]:
day_week_avg = (
    util_day_ofweek
    .groupby('DAY_OF_WEEK', observed=False)['UTILIZATION_RATE_%']
    .mean()
    .reset_index()
)

weekday_in_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_week_avg['DAY_OF_WEEK'] = pd.Categorical(day_week_avg['DAY_OF_WEEK'], categories=weekday_in_order, ordered=True)
day_week_avg = day_week_avg.sort_values('DAY_OF_WEEK')

fig2 = px.bar(
    day_week_avg,
    x='DAY_OF_WEEK',
    y='UTILIZATION_RATE_%',
    text=day_week_avg['UTILIZATION_RATE_%'].round(1).astype(str) + '%',
    title='Average Utilization Rate by Days of the Week',
    labels={'UTILIZATION_RATE_%': 'Average Utilization Rate (%)', 'DAY_OF_WEEK': 'Days of the Week'}
)

fig2.update_layout(template='simple_white')
fig2.show()